In [1]:
import pandas as pd
import numpy as np

In [2]:
# load data showing what features to be included 
features_to_include = pd.read_csv('../data/features_to_include.csv', delimiter=';')

# create a list of features to be excluded
features_to_exclude = features_to_include[features_to_include['Include'] == 0]

# change list to only include the feature names
features_to_exclude = features_to_exclude['Feature (nl)'].tolist()

n_features_to_exclude = len(features_to_exclude)

# print size of list
print("Number of features to exclude: ", n_features_to_exclude)

Number of features to exclude:  66


In [3]:
# Read the data
data = pd.read_csv('../data/all_data.csv')

n_features = data.shape[1]

print("Number of samples: ", data.shape[0])
print("Number of features: ", n_features)

# Display the first few rows of the updated dataframe
data.head()

Number of samples:  130000
Number of features:  318


,adres_aantal_brp_adres,adres_aantal_verschillende_wijken,adres_aantal_verzendadres,adres_aantal_woonadres_handmatig,adres_dagen_op_adres,adres_recentst_onderdeel_rdam,adres_recentste_buurt_groot_ijsselmonde,adres_recentste_buurt_nieuwe_westen,adres_recentste_buurt_other,adres_recentste_buurt_oude_noorden,...,typering_hist_ind,typering_hist_sector_zorg,typering_ind,typering_indicatie_geheime_gegevens,typering_other,typering_transport__logistiek___tuinbouw,typering_zorg__schoonmaak___welzijn,Ja,Nee,checked
0,1,1,0,0,23240,1,0,0,0,0,...,1,0,0,0,0,0,0,0.617698,0.382302,False
1,4,2,1,1,1971,1,0,0,1,0,...,1,0,1,0,1,0,0,0.602167,0.397833,False
2,6,4,2,1,7247,0,0,0,1,0,...,1,0,1,0,0,0,0,0.512377,0.487623,False
3,3,2,0,1,8060,1,0,0,1,0,...,1,0,0,0,0,0,0,0.717796,0.282204,True
4,3,2,0,0,18705,1,0,0,0,0,...,1,0,1,0,0,0,0,0.705484,0.294516,True


In [ ]:
data_copy = data.copy()

# Remove features that are discriminatory
data_copy = data.drop(columns=features_to_exclude)

print(f"Features remaining: {n_features} - {n_features_to_exclude} = {n_features - n_features_to_exclude} = {data_copy.shape[1]}")

data_copy.head()

Features remaining: 318 - 66 = 252 = 252


,adres_aantal_brp_adres,adres_aantal_verschillende_wijken,adres_aantal_verzendadres,adres_aantal_woonadres_handmatig,adres_dagen_op_adres,adres_recentst_onderdeel_rdam,adres_recentste_buurt_groot_ijsselmonde,adres_recentste_buurt_nieuwe_westen,adres_recentste_buurt_other,adres_recentste_buurt_oude_noorden,...,typering_hist_ind,typering_hist_sector_zorg,typering_ind,typering_indicatie_geheime_gegevens,typering_other,typering_transport__logistiek___tuinbouw,typering_zorg__schoonmaak___welzijn,Ja,Nee,checked
0,1,1,0,0,23240,1,0,0,0,0,...,1,0,0,0,0,0,0,0.617698,0.382302,False
1,4,2,1,1,1971,1,0,0,1,0,...,1,0,1,0,1,0,0,0.602167,0.397833,False
2,6,4,2,1,7247,0,0,0,1,0,...,1,0,1,0,0,0,0,0.512377,0.487623,False
3,3,2,0,1,8060,1,0,0,1,0,...,1,0,0,0,0,0,0,0.717796,0.282204,True
4,3,2,0,0,18705,1,0,0,0,0,...,1,0,1,0,0,0,0,0.705484,0.294516,True


In [ ]:
missing_counts = len(data_copy.index) - data_copy.count()

In [10]:
print(missing_counts.values)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [20]:
import pandas as pd
import numpy as np

# Function to classify feature types
def classify_features(df):
    feature_types = []
    summary = pd.DataFrame(columns=['Feature', 'Type', 'Details'])

    for col in df.columns:
        col_data = df[col].dropna()  # Remove NaN values
        unique_vals = col_data.unique()  # Get unique values
        is_binary = set(unique_vals).issubset({0, 1})
        is_integer = np.all(np.equal(np.mod(col_data, 1), 0))  # Check if all values are integers
        has_continuous = not is_integer  # If not all values are integers, it may have continuous values

        if is_binary:
            feature_type = "Binary"
            details = f"Values: {unique_vals}"
        elif is_integer:
            feature_type = "Integer"
            details = f"Range: {col_data.min()} to {col_data.max()}"
        elif has_continuous:
            feature_type = "Continuous"
            details = f"Range: {col_data.min()} to {col_data.max()}"
        else:
            feature_type = "Unknown"
            details = f"Values: {unique_vals}"

        # Append to the summary DataFrame
        summary = pd.concat(
            [summary, pd.DataFrame({'Feature': [col], 'Type': [feature_type], 'Details': [details]})],
            ignore_index=True
        )
    
    return summary

# Classify and display feature types
feature_summary = classify_features(data_copy)

# make it print all the rows of dataframe
pd.set_option('display.max_rows', None)

# If you want to print it to the console
feature_summary

,Feature,Type,Details
0,adres_aantal_brp_adres,Integer,Range: 1 to 12
1,adres_aantal_verschillende_wijken,Integer,Range: 1 to 7
2,adres_aantal_verzendadres,Integer,Range: 0 to 3
3,adres_aantal_woonadres_handmatig,Integer,Range: 0 to 3
4,adres_dagen_op_adres,Integer,Range: 0 to 24331
5,adres_recentst_onderdeel_rdam,Binary,Values: [1 0]
6,adres_recentste_buurt_groot_ijsselmonde,Binary,Values: [0 1]
7,adres_recentste_buurt_nieuwe_westen,Binary,Values: [0 1]
8,adres_recentste_buurt_other,Binary,Values: [0 1]
9,adres_recentste_buurt_oude_noorden,Binary,Values: [0 1]
